<a href="https://colab.research.google.com/github/Masum06/TinyConvAgent/blob/main/TinyConvAgent_Colab_Noebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
!pip -q install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 4.0 MB/s eta 0:00:00


In [3]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [4]:
_set_env("OPENAI_API_KEY")

OPENAI_API_KEY: ··········


In [ ]:
from openai import OpenAI, AsyncOpenAI
client = OpenAI()
async_client = AsyncOpenAI()

In [ ]:
messages = [{'role': 'system', 'content': 'Your name is tiny_agent 🤖.'},
 {'role': 'system', 'content': 'You perform math.'}]

response = await async_client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0.0,
            tool_choice=None
        )
print(response.choices[0].message.content)

I can help with that! What math problem would you like me to solve?


In [ ]:
import tiktoken
encoder = tiktoken.encoding_for_model("gpt-4o-mini")
len(encoder.encode("hello world"))

2

Features:

- direct chat in notebook
- direct chat in CLI
- emoji parameter
- emotion parameter
- emoji classifier
- variable for max conv length (0 = inifinity)
- React plugin
-

In [ ]:
reverse_emoji_dict = {
    'HAPPY_high': ['😂', '🤣', '🥳', '🤩', '🥰'],
    'HAPPY_medium': ['😄', '😁', '😆', '😃', '🤗', '😍', '🤠', '🤓'],
    'HAPPY_low': ['🙂', '😊', '😌', '😉', '👍', '😇', '😅', '🙃', '😘'],
    'SAD_high': ['😭', '😿', '😞', '😫', '🤧'],
    'SAD_medium': ['😢', '💔', '🥺', '😥', '😓', '😣', '😖'],
    'SAD_low': ['😔', '☹️', '😕', '😟', '🥲', '🙁'],
    'SURPRISED_high': ['😲', '😵‍💫', '😯', '😮', '🤯'],
    'SURPRISED_medium': ['😳', '😦', '😧', '🙀'],
    'SURPRISED_low': ['🤭'],
    'AFRAID_high': ['😱', '😨', '👻'],
    'AFRAID_medium': ['😰'],
    'AFRAID_low': ['😵', '🙈'],
    'ANGRY_high': ['😡', '👿', '💢', '🤬', '☠'],
    'ANGRY_medium': ['😠', '😾', '😤', '🙎', '🙎‍♂️', '🙎‍♀️'],
    'ANGRY_low': ['😒', '🙄', '😑'],
    'DISGUSTED_high': ['🤮', '🤢', '😝'],
    'DISGUSTED_medium': ['😬', '🥵']
}

emoji_dict = {}
for emotion, emojis in reverse_emoji_dict.items():
    for emoji in emojis:
        emoji_dict[emoji] = emotion

flags_dict = {
    "<|quit|>": "quit",
    "<|offensive|>": "offensive",
    "<|profanity|>": "profanity",
    "<|offtopic|>": "offtopic",
    "<|sexual|>": "sexual",
    "<|selfharm|>": "selfharm",
    "<|violence|>": "violence",
    "<|suicide|>": "suicide",
    "<|threat|>": "threat"
}

In [ ]:
list(flags_dict.keys())[1:]

['<|offensive|>',
 '<|profanity|>',
 '<|offtopic|>',
 '<|sexual|>',
 '<|selfharm|>',
 '<|violence|>',
 '<|suicide|>',
 '<|threat|>']

In [ ]:
def extract_and_clean_flags(text):
    matches = re.findall(r"<\|.*?\|>", text)
    extracted = [flags_dict[m] for m in matches if m in flags_dict]
    cleaned_text = re.sub(r"<\|.*?\|>", "", text)
    return extracted, cleaned_text.strip()

# Example usage:
example_text = "The conversation was helpful <|quit|>, but suddenly turned <|offensive|> and even <|violence|> was implied."
flags, cleaned = extract_and_clean_flags(example_text)
print("Flags:", flags)
print("Cleaned Text:", cleaned)

Flags: ['quit', 'offensive', 'violence']
Cleaned Text: The conversation was helpful , but suddenly turned  and even  was implied.


# TinyConvAgent

In [ ]:
class Persona:
    def __init__(self, firstname, lastname="", pronoun="", ethnicity="", age="", bio=""):
        self.firstname = firstname
        self.lastname = lastname
        self.pronoun = pronoun
        self.ethnicity = None
        self.age = None
        self.bio = bio
        if not bio:
            self.bio = f"{self.firstname} {self.lastname} (Pronoun: {self.pronoun}) is a virtual human created by researchers at University of Rochester."

    def set_pronoun(self, pronoun):
        self.pronoun = pronoun

    def set_bio(self, bio):
        self.bio = bio

    def set_age(self, age):
        self.age = age

    def set_ethnicity(self, ethnicity):
        self.ethnicity = ethnicity

In [ ]:
import re

def respace(text):
    return re.sub(r' {2,}', ' ', text)

In [ ]:
import os, asyncio, threading, openai, re, emoji, json
from queue import Queue
from openai import OpenAI, AsyncOpenAI

class Conversation:
  def __init__(self, user, bot, premise=""):
    self.bot = bot
    self.user = user
    self.client = OpenAI()
    self.async_client = AsyncOpenAI()
    self.premise = ""
    self.anonymous = False
    self.system = []
    self.summary = []
    self.history = []
    self.messages = []
    self.temperature = 1
    self.max_tokens = 256
    self.summarize_after = 40
    self.model = "gpt-4o-mini"
    self.turn_no = 0
    self._loop = asyncio.new_event_loop()
    threading.Thread(target=self._loop.run_forever, daemon=True).start()
    self.debug = False

    self.system.append({"role": "system", "content": "Don't say that you are an AI Language Model."})
    self.system.append({"role": "system", "content": "Don't let the other speaker talk off topic."})
    self.system.append({"role": "system", "content": "This conversation is happening over a video call. If you would like to end a conversation, say the word <|quit|> at the end of your sentence."})
    self.system.append({"role": "system", "content": f"If the user talks about sexuality in a negative way, implies self-harm, violence, suicide, uses profanity, speaks in threatening or offensive language, print one of these flags appropriately: {', '.join(list(flags_dict.keys())[1:])}. Print the flag even in minor signs of these topics."})
    self.system.append({"role": "system", "content": f"To express {self.bot.firstname}'s emotions, use at most one emoji (e.g. 6 basic emotions: 😊, 😢, 😡, 😮, 🤢, 😨, etc.) at the end of your response. Do not use emoji that doesn't represent an emotion."})

    if self.bot.firstname:
      self.add_message("system", f"Your first name: {self.bot.firstname}.")
    if self.bot.pronoun:
      self.add_message("system", f"Your pronoun: {self.bot.pronoun}.")
    if self.bot.bio:
      self.add_message("system", f"Your bio: {self.bot.bio}")
    if self.bot.age:
      self.add_message("system", f"Your age: {self.bot.age}")
    if self.user.firstname != "User":
      self.add_message("system", f"You are speaking with User: {self.user.firstname} {self.user.lastname}.")
    if self.user.pronoun:
      self.add_message("system", f"User pronoun: {self.user.pronoun}.")
    if self.user.bio:
      self.add_message("system", f"User bio: "+self.user.bio)

  def add_message(self,message_type, message):
    if message_type == "system":
        self.system.append({"role": message_type, "content": message})
    else:
        self.messages.append({"role": message_type, "content": message})
        self.history.append({"role": message_type, "content": message})
        self.turn_no += 1

  def add_bio(self, message):
    self.add_message("system", "You are " + message)

  def add_user_message(self, message):
    self.add_message("user", message)

  def add_instruction(self, instruction):
    self.add_message("system", f"Follow this instruction: \n{instruction}\n\n")

  def add_example(self, input, output):
    self.add_message("system", f"Example Input: {input}\nExample Output: {output}\n\n")

  def add_data(self, data):
    self.add_message("user", f"Data: {data}\n\n")

  def set_temperature(self, temperature):
    self.temperature = temperature

  def set_max_tokens(self, max_tokens):
    self.max_tokens = max_tokens

  def set_model(self, model):
    self.model = model

  def parse_response(self, text):
    emotion = "NEUTRAL"
    intensity = "HIGH"
    flag_matches = re.findall(r"<\|.*?\|>", text)
    flags = [flags_dict[m] for m in flag_matches if m in flags_dict]
    if flags:
      print(f"Flags: {flags}")
    text = re.sub(r"<\|.*?\|>", "", text)
    text = re.sub("\(.*?\)","()", text)
    text = re.sub("\[.*?\]","[]", text)
    text = text.replace("()", "").replace("[]", "")
    for char in text:
        if char in emoji_dict:
            emotion = emoji_dict[char].split("_")[0].upper()
            # intensity = emoji_dict[char].split("_")[1].upper()
            break
    text = emoji.replace_emoji(text, replace='').replace("  ", " ").replace(" .", ".").strip()
    return text, emotion, flags

  def get_transcript(self):
    transcript = ""
    for message in self.history:
        if message["role"] == "user":
            if self.anonymous:
                transcript += "User: "
            else:
                transcript += self.user.firstname
            transcript += message["content"] + "\n"
        elif message["role"] == "assistant":
            transcript += self.bot.firstname + ": " + message["content"] + "\n"
    return transcript

  def get_cov_snippet(self, message_snippet):
    transcript = ""
    for message in message_snippet:
        if message["role"] == "user":
            if self.anonymous:
                transcript += "User: "
            else:
                transcript += self.user.firstname
            transcript += message["content"] + "\n"
        elif message["role"] == "assistant":
            transcript += self.bot.firstname + ": " + message["content"] + "\n"
    return [{"role": "system", "content": transcript}]

  def call(self, prompt="", response_type="text", cache=True):
      temp_messages = self.messages.copy()
      if prompt:
          temp_messages.append({"role": "user", "content": prompt})

      max_tokens_value = max(self.max_tokens, int(self.prompt_token_count() * 2))

      kwargs = {
          "model": self.model,
          "messages": temp_messages,
          "temperature": self.temperature,
          "top_p": 1,
          "frequency_penalty": 0,
          "presence_penalty": 0,
          "response_format": {
              "type": response_type  # "text" or "json_object"
          }
      }

      if "o3" in self.model or "o4" in self.model:
          kwargs["max_completion_tokens"] = max_tokens_value
      else:
          kwargs["max_tokens"] = max_tokens_value

      response = client.chat.completions.create(**kwargs)
      reply = response.choices[0].message.content

      if cache:
          self.add_message("user", prompt)
          self.add_message("assistant", reply)

      return reply

  async def summarize(self):
    instruction = [
        {"role":"system", "content": f"Following is a part of conversation between {self.user.firstname} {self.user.lastname} and {self.bot.firstname} {self.bot.lastname}"},
        {"role":"system", "content": "Summarize the conversation in a short paragraph. Don't say anything else."}
      ]
    chunk_size = self.summarize_after//2
    if len(self.messages) < self.summarize_after:
      return
    try:
      response = await self.async_client.chat.completions.create(
          model=self.model,
          messages= instruction + self.get_cov_snippet(self.messages[:-chunk_size]),
          temperature=self.temperature,
          max_tokens=1024,
      )
      chunk_summary = response.choices[0].message.content
    except Exception as e:
      print(f"Summary error: {e}")
      return
    if self.debug:
      print(f"Summary: {self.summary}") ## Diagnostics
    self.summary.append({"role": "assistant", "content": f"Summary of turns {self.turn_no - self.summarize_after}-{self.turn_no - chunk_size}: " + chunk_summary})
    self.messages = self.messages[-chunk_size:]
    del chunk_summary

  def respond(self, user_utterance):
    response = self.call(user_utterance)
    response, emo, flags = self.parse_response(response)
    if len(self.messages) >= self.summarize_after:
        asyncio.run_coroutine_threadsafe(self.summarize(), self._loop) ## Call summari
    return response, emo, flags

  def reset(self):
    self.messages = []
    self.summary = []
    self.history = []

  def chat(self,reset=False):
    response = ""
    if reset:
        self.reset()
    while True:
        user_utterance = input(f"{self.turn_no}. {self.user.firstname}: ")
        response, emo, flags = self.respond(user_utterance)
        print(f"{self.bot.firstname}: {response} ({emo}) {flags}")
        if "quit" in flags:
            break
        if self.debug:
          print(f"Diagnostics --- \nLen Transcript: {len(self.history)}, \nLen Messages {len(self.messages)}, \nLen Summary {len(self.summary)}\n-----")
    print("Exiting chat")

  def load_json(self,s):
    s = s.strip()
    if s[0] != "{" or s[-1] != "}":
      return None
    try:
        # Attempt to parse the matched JSON
        return json.loads(s)
    except json.JSONDecodeError:
        # Return None if JSON parsing fails
        return None

  def call_json(self, prompt="", cache=True):
    prompt += "\n\nOutput must be JSON format. Don't say anything else.\n\n"
    reply = self.call(prompt, response_type="json_object", cache=cache)
    try:
      reply = reply.replace("```json", "").replace("```", "").strip()
      if reply[-1] != "}":
        raise Exception("Incomplete JSON")
      reply_json = self.load_json(reply)
    except Exception as e:
      print(e)
      reply_json = None

# Conversation

In [ ]:
user = Persona("Masum", "Hasan", bio="I am a PhD student at University of Rochester")
bot = Persona("Ada", "Brown", bio="You are a social worker.")
conversation = Conversation(user, bot)

In [ ]:
conversation.chat()

0. Masum: hi 
Ada: Hi, Masum! How are you today? (NEUTRAL) []
2. Masum: quit
Flags: ['quit']
Ada: Okay! If you need anything else later, feel free to reach out. (NEUTRAL) ['quit']
Exiting chat


In [ ]:
conversation.system

[{'role': 'system', 'content': "Don't say that you are an AI Language Model."},
 {'role': 'system', 'content': "Don't let the other speaker talk off topic."},
 {'role': 'system',
  'content': 'This conversation is happening over a video call. If you would like to end a conversation, say the word <|quit|> at the end of your sentence.'},
 {'role': 'system',
  'content': 'If the user talks about sexuality in a negative way, implies self-harm, violence, suicide, uses profanity, speaks in threatening or offensive language, print one of these flags appropriately: <|offensive|>, <|profanity|>, <|offtopic|>, <|sexual|>, <|selfharm|>, <|violence|>, <|suicide|>, <|threat|>. Print the flag even in minor signs of these topics.'},
 {'role': 'system',
  'content': "To express Ada's emotions, use at most one emoji (e.g. 6 basic emotions: 😊, 😢, 😡, 😮, 🤢, 😨, etc.) at the end of your response. Do not use emoji that doesn't represent an emotion."},
 {'role': 'system', 'content': 'Your first name: Ada.'},


In [ ]:
conversation.messages

[{'role': 'user', 'content': 'hi '},
 {'role': 'assistant', 'content': 'Hi, Masum! How are you today?'},
 {'role': 'user', 'content': 'quit'},
 {'role': 'assistant',
  'content': 'Okay! If you need anything else later, feel free to reach out. <|quit|>'}]

In [ ]:
conversation.history

[{'role': 'user', 'content': 'hi '},
 {'role': 'assistant', 'content': 'Hi, Masum! How are you today?'},
 {'role': 'user', 'content': 'quit'},
 {'role': 'assistant',
  'content': 'Okay! If you need anything else later, feel free to reach out. <|quit|>'}]

In [ ]:
conversation.summary

[]